import

In [1]:
import pandas as pd
import time,datetime
import numpy as np
from pandas import *
from informs import portfolio
import cmath
import time
import class_informs as inform

데이터 조정치

In [2]:
#timeseries data
all_data=pd.read_csv('Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()



dic_data = {k: v for k, v in all_data.groupby('date')}

using_dic = dic_data[pd.to_datetime(date_list[0])]



asset_list = []
for i in using_dic["sedol"]:
    asset_list.append(i)


dic_sedol_as = {using_dic["sedol"][i] : 10000*using_dic["as"][i] for i in using_dic.index}

dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }
dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

for i in using_dic.index:
    dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
    dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])


#risk:cov_mat
def risk(date_str):
    risk_data=pd.read_csv('cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    return risk_data,risk_sedol,risk_mat
risk_data,risk_sedol,risk_mat=risk(date_list[0])



sedol_var_list =[]

for i in risk_sedol:
    sedol_var_list.append("d"+str(i))

for i in risk_sedol:
    sedol_var_list.append("q"+str(i))



sedol_var_list.append("assum")
alpha = []


for i in risk_sedol:

    alpha.append(-1*dic_sedol_as[i])

qmat=[]





for i in range(len(risk_mat)):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(10000*j)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)

for i in range(len(risk_mat)):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)

for i in range(1):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)






q_con1 = []
q_con2 = []
q_val = []


for i in range(len((risk_mat[0]))):
    for j in range(len(risk_mat[0])):
        if j >= i:
            q_con1.append(i)
            q_con2.append(j)
            if i == j:
                ex_list = list(risk_mat[i])
                q_val.append(0.5*ex_list[j]*10000)
            else:
                ex_list = list(risk_mat[i])
                q_val.append(ex_list[j]*10000)



Q_con = []
Q_con.append(q_con1)
Q_con.append(q_con2)
Q_con.append(q_val)



# dic_result = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000)

함수 정의

In [11]:
def cal_obj(sol, risk_mat, alpha) :
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, alpha)
    obj = s_r+s_a
    return obj*10000


def select_candi(sol) :
    save_sub = [0]*len(alpha)
    save_add = [0]*len(alpha)
    diff_sub = [0]*len(alpha)
    diff_add = [0]*len(alpha)
    candi = []
#     avgs = cal_obj(sol, risk_mat, alpha)
    
    for i in range(len(sol)) :
        if(sol[i] > 0) :
            w = sol[i]
            save_sub[i] = w
            diff_sub[i] = cal_obj(save_sub, risk_mat, alpha)
            np_su = np.array(diff_sub)
            
            if min(np_su[np_su>0]) == diff_sub[i] :
                for j in range(len(sol)) :
                    if sol[j] == 0 :
                        save_add[j] = w
                        diff_add[j] = cal_obj(save_add, risk_mat, alpha)
                        save_add[j] = 0
            save_sub[i] = 0

    # print(diff_sub)
    # print(max(diff_sub))
    # print(diff_sub.index((max(diff_sub))))

    np_ad = np.array(diff_add)
    # print(diff_add)
    # print(min(x[x>0]))
    # print(diff_add.index((min(x[x>0]))))

    
    
    if sol[diff_sub.index((min(np_su[np_su>0])))] == sol[diff_add.index((min(np_ad[np_ad>0])))] :
        sol[diff_add.index((min(np_ad[np_ad>min(np_ad[np_ad>0])])))] = 0.1
    else :
        sol[diff_add.index((min(np_ad[np_ad>0])))] = 0.1
        
    sol[diff_sub.index((min(np_su[np_su>0])))] = 0

    for val in sol :
        if val > 0 :
            candi.append(sol.index(val))
            
    return candi





def update(sol) :
    start_time = time.time()
    ex = inform.informs(10000)
    ex.set_omega(risk_mat)
    
    sol_list = []
    for k in risk_sedol :
        sol_list.append(sol[k])
    result_list = []
    
    while(True) :
        if time.time() - start_time > 200:
            break
        con = select_candi(sol_list)
        ex.set_con(cons=con)
        sol_list2 = []
        try:
            sol2 = ex.solve(0.01)
            if sol2 != 0:
                for k in risk_sedol :
                    sol_list2.append(sol2[0][k])
                sol_list = sol_list2
                print(sol_list)
                result_list.append(sol2[2])
            else:
                print("infeasible solution")
        except:
            print("no solution")
            
    return result_list, sol_list

초기 cplex 실행

In [7]:
sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000, time_init=30.0)[0]

CPXPARAM_TimeLimit                               30
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 2 times.
MIQCP Presolve eliminated 1486 rows and 2 columns.
MIQCP Presolve modified 1476 coefficients.
Aggregator did 58 substitutions.
Reduced MIQCP has 1941 rows, 2839 columns, and 197656 nonzeros.
Reduced MIQCP has 492 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQCP has 2 quadratic constraints.
Presolve time = 0.16 sec. (431.54 ticks)
Probing time = 0.03 sec. (21.83 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.03 sec. (39.21 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0  

선택 자산 update

In [18]:
result = update(dfd)

print("objective value 변화")
print(result[0])
# print("portfolio's weight")
# print(result[1])

ValueError: min() arg is an empty sequence

In [13]:
dfd={'2000019': 0.0,
 '2002305': 0.0,
 '2002479': 0.0,
 '2005973': 0.0,
 '2007849': 0.0039896692082423604,
 '2008154': 0.0,
 '2011602': 0.0,
 '2017213': 0.0,
 '2018175': 0.0,
 '2019479': 0.0,
 '2019952': 0.0,
 '2020459': 0.0,
 '2021690': 0.0,
 '2023607': 0.0,
 '2023737': 0.010691944567940037,
 '2023748': 0.0,
 '2024020': 0.0,
 '2026082': 0.0,
 '2026242': 0.0,
 '2026361': 0.0,
 '2027104': 0.0,
 '2027342': 0.0,
 '2028572': 0.0,
 '2029809': 0.0,
 '2032067': 0.0,
 '2032380': 0.0,
 '2033004': 0.0,
 '2034494': 0.0,
 '2034524': 0.0,
 '2038430': 0.0,
 '2039831': 0.0,
 '2041364': 0.0,
 '2043694': 0.0,
 '2043962': 0.0,
 '2044545': 0.0,
 '2046251': 0.018417262520140518,
 '2046552': 0.0,
 '2047317': 0.0,
 '2048804': 0.024950686485023615,
 '2050832': 0.0,
 '2057059': 0.0,
 '2065159': 0.0038942869231824717,
 '2065308': 0.0,
 '2065955': 0.0,
 '2066408': 0.0,
 '2066721': 0.0033493087018528125,
 '2068921': 0.0,
 '2069065': 0.0,
 '2073022': 0.0,
 '2073408': 0.0,
 '2077905': 0.0,
 '2081779': 0.0,
 '2084637': 0.0,
 '2085102': 0.0,
 '2085878': 0.0,
 '2086138': 0.0,
 '2087807': 0.0,
 '2090173': 0.0,
 '2090571': 0.0,
 '2094670': 0.0,
 '2098597': 0.0,
 '2098876': 0.0,
 '2100920': 0.035482906950411904,
 '2101503': 0.013837343734514968,
 '2101967': 0.0,
 '2105505': 0.0,
 '2108601': 0.0,
 '2109002': 0.009116219839156621,
 '2110703': 0.0,
 '2113434': 0.0,
 '2126335': 0.0,
 '2138158': 0.0,
 '2142784': 0.0,
 '2142922': 0.0,
 '2146838': 0.0,
 '2149309': 0.0,
 '2154927': 0.0,
 '2160753': 0.0,
 '2162845': 0.0,
 '2170473': 0.0,
 '2175672': 0.0,
 '2180201': 0.0,
 '2182348': 0.0,
 '2182553': 0.0,
 '2182779': 0.0,
 '2183244': 0.0,
 '2183330': 0.0,
 '2185046': 0.0,
 '2190385': 0.048809514225985853,
 '2192295': 0.0,
 '2195722': 0.0,
 '2196479': 0.0083309444482847876,
 '2196888': 0.0,
 '2197137': 0.0,
 '2198163': 0.0,
 '2199326': 0.0016307819638546708,
 '2202536': 0.0,
 '2204026': 0.0,
 '2206657': 0.0,
 '2209106': 0.0,
 '2210614': 0.0,
 '2212870': 0.0,
 '2214832': 0.0,
 '2215460': 0.0,
 '2216850': 0.0,
 '2218447': 0.0,
 '2219224': 0.0,
 '2220884': 0.0,
 '2224701': 0.0,
 '2229676': 0.0,
 '2232696': 0.0,
 '2232793': 0.0,
 '2235822': 0.0,
 '2236911': 0.0,
 '2240202': 0.0,
 '2246288': 0.0,
 '2249629': 0.0,
 '2250687': 0.0,
 '2250870': 0.0,
 '2252058': 0.0,
 '2256511': 0.0,
 '2257019': 0.0,
 '2259101': 0.0,
 '2259468': 0.0,
 '2261203': 0.0,
 '2261526': 0.0,
 '2262314': 0.0,
 '2269768': 0.0,
 '2270726': 0.0,
 '2272205': 0.0,
 '2278407': 0.0,
 '2279002': 0.0,
 '2279303': 0.0,
 '2280220': 0.0,
 '2284761': 0.0,
 '2289874': 0.0,
 '2293819': 0.0094592233124947932,
 '2295172': 0.0,
 '2295677': 0.0,
 '2295945': 0.0050969447611214349,
 '2297907': 0.0,
 '2298386': 0.0,
 '2299011': 0.0,
 '2300601': 0.0,
 '2304227': 0.0,
 '2305844': 0.0,
 '2310194': 0.0040369538333418446,
 '2312736': 0.0,
 '2313405': 0.0,
 '2317087': 0.0,
 '2318024': 0.0,
 '2319146': 0.0,
 '2319157': 0.0,
 '2320524': 0.0,
 '2326618': 0.0,
 '2328915': 0.0,
 '2331225': 0.0,
 '2333889': 0.0,
 '2334150': 0.037115746773899151,
 '2336747': 0.0010000006008030276,
 '2340168': 0.0,
 '2341484': 0.0,
 '2341860': 0.0,
 '2342034': 0.0,
 '2344331': 0.0027734603549230542,
 '2345022': 0.0,
 '2346508': 0.0,
 '2350684': 0.019841728004404879,
 '2352118': 0.0,
 '2360326': 0.0,
 '2365161': 0.0,
 '2365804': 0.0051101011387761214,
 '2367026': 0.0,
 '2367480': 0.0,
 '2367598': 0.0,
 '2369174': 0.0,
 '2377809': 0.0,
 '2378200': 0.0,
 '2378534': 0.0,
 '2379504': 0.0,
 '2380498': 0.0,
 '2380863': 0.0,
 '2404569': 0.0,
 '2405302': 0.0,
 '2407966': 0.0,
 '2411053': 0.0,
 '2411138': 0.0,
 '2413758': 0.0,
 '2414580': 0.0,
 '2417868': 0.0,
 '2417987': 0.0,
 '2419240': 0.0,
 '2422442': 0.0,
 '2422806': 0.0,
 '2428008': 0.0,
 '2429090': 0.0,
 '2431846': 0.00437716327086996,
 '2433842': 0.0,
 '2434209': 0.0,
 '2440637': 0.0,
 '2445063': 0.0,
 '2445966': 0.0,
 '2455965': 0.0,
 '2457552': 0.0,
 '2459020': 0.0,
 '2463247': 0.0,
 '2464165': 0.0,
 '2464972': 0.0,
 '2465254': 0.0,
 '2466321': 0.0,
 '2471789': 0.0,
 '2475833': 0.0,
 '2476193': 0.0,
 '2476803': 0.0,
 '2477152': 0.0,
 '2480138': 0.014724378257259595,
 '2480677': 0.0,
 '2484088': 0.0,
 '2485070': 0.0,
 '2486813': 0.020431210622934178,
 '2490911': 0.001000000104928012,
 '2491594': 0.0,
 '2491839': 0.0,
 '2496113': 0.0,
 '2497406': 0.0,
 '2498993': 0.0,
 '2510615': 0.0080493317514140576,
 '2510682': 0.0,
 '2510723': 0.0,
 '2511328': 0.013280106397184303,
 '2511908': 0.0,
 '2511920': 0.0,
 '2516152': 0.054033737978393621,
 '2516378': 0.0,
 '2516839': 0.0,
 '2517382': 0.0,
 '2520153': 0.0,
 '2522096': 0.038642175284899397,
 '2523022': 0.0,
 '2523044': 0.0,
 '2526117': 0.0,
 '2536763': 0.031828674188536216,
 '2542049': 0.0,
 '2547044': 0.0,
 '2548616': 0.0,
 '2550161': 0.0,
 '2550707': 0.0,
 '2567741': 0.0,
 '2570200': 0.0,
 '2570545': 0.0,
 '2572303': 0.0,
 '2573209': 0.0,
 '2575896': 0.0,
 '2576350': 0.0,
 '2577609': 0.0,
 '2578516': 0.0,
 '2580986': 0.0,
 '2586122': 0.0,
 '2588173': 0.042098050680030488,
 '2588184': 0.0,
 '2589198': 0.0,
 '2594437': 0.0,
 '2595708': 0.0,
 '2599142': 0.0,
 '2604206': 0.0,
 '2605555': 0.0,
 '2611206': 0.0,
 '2614807': 0.0,
 '2615468': 0.0,
 '2624356': 0.0,
 '2624486': 0.0,
 '2625047': 0.0,
 '2626921': 0.0,
 '2630643': 0.0,
 '2632003': 0.0,
 '2632650': 0.0,
 '2634805': 0.0,
 '2635701': 0.0,
 '2636607': 0.0,
 '2639509': 0.0,
 '2640147': 0.016040344083995225,
 '2641827': 0.0,
 '2641894': 0.0098919409133483033,
 '2645409': 0.0,
 '2646015': 0.0,
 '2648668': 0.0,
 '2648806': 0.0,
 '2649100': 0.05946163813106399,
 '2650748': 0.0,
 '2650793': 0.0,
 '2651086': 0.0,
 '2654320': 0.0,
 '2654461': 0.0,
 '2655408': 0.053156378157092787,
 '2655981': 0.0,
 '2661568': 0.0,
 '2665861': 0.0,
 '2668804': 0.0,
 '2670519': 0.0,
 '2671501': 0.0,
 '2672689': 0.0,
 '2674458': 0.0020577482154922862,
 '2680303': 0.0,
 '2680905': 0.0,
 '2681403': 0.0,
 '2681511': 0.0,
 '2681801': 0.0,
 '2684703': 0.0,
 '2685007': 0.0,
 '2685717': 0.0,
 '2689560': 0.0,
 '2690506': 0.022223168209551482,
 '2692397': 0.0,
 '2692632': 0.0,
 '2692665': 0.044501555951956429,
 '2695921': 0.0,
 '2696838': 0.0,
 '2698470': 0.0,
 '2699291': 0.01750601892598657,
 '2701271': 0.0,
 '2702337': 0.0,
 '2702791': 0.0,
 '2704407': 0.066037301081239175,
 '2705024': 0.0,
 '2705648': 0.0,
 '2707677': 0.0,
 '2708841': 0.0,
 '2714923': 0.0,
 '2717320': 0.0,
 '2717483': 0.0,
 '2732635': 0.0,
 '2736035': 0.0,
 '2746829': 0.0,
 '2754060': 0.0,
 '2758051': 0.029529898918561705,
 '2760669': 0.0,
 '2765802': 0.0,
 '2766805': 0.0,
 '2767165': 0.0,
 '2767228': 0.0,
 '2767381': 0.0,
 '2769503': 0.0,
 '2769796': 0.0,
 '2778844': 0.0,
 '2779201': 0.0,
 '2779397': 0.0,
 '2783815': 0.0,
 '2786159': 0.0047230904916796546,
 '2790611': 0.0,
 '2795393': 0.0,
 '2803014': 0.0,
 '2804211': 0.0,
 '2807492': 0.0040040849326287258,
 '2807566': 0.0,
 '2812452': 0.0,
 '2814641': 0.0,
 '2816409': 0.0,
 '2818740': 0.0,
 '2819118': 0.0,
 '2820563': 0.0,
 '2822600': 0.0,
 '2824770': 0.0,
 '2829515': 0.0,
 '2829601': 0.0,
 '2830904': 0.0,
 '2831543': 0.0,
 '2831811': 0.0,
 '2838555': 0.0,
 '2841489': 0.0,
 '2842040': 0.0,
 '2842255': 0.0,
 '2852533': 0.0,
 '2853688': 0.0,
 '2859868': 0.0,
 '2859954': 0.0090905274923826357,
 '2860990': 0.0,
 '2861078': 0.0,
 '2867913': 0.0,
 '2868165': 0.03676570379975598,
 '2871505': 0.0,
 '2880255': 0.0,
 '2880501': 0.0,
 '2881537': 0.0,
 '2883395': 0.0,
 '2884183': 0.0,
 '2884224': 0.0,
 '2885409': 0.0,
 '2885700': 0.0,
 '2885937': 0.0,
 '2886907': 0.0,
 '2892045': 0.0,
 '2892090': 0.0,
 '2896713': 0.0,
 '2903550': 0.011819691880953784,
 '2909730': 0.0,
 '2910970': 0.0,
 '2914734': 0.0,
 '2915500': 0.0,
 '2917766': 0.0,
 '2922544': 0.0,
 '2928683': 0.0,
 '2931205': 0.0,
 '2931595': 0.0,
 '2933632': 0.005978490100746232,
 '2936921': 0.0,
 '2937667': 0.017763130043368217,
 '2937689': 0.0,
 '2941925': 0.0,
 '2942188': 0.0,
 '2947956': 0.0091650280070993701,
 '2954019': 0.0,
 '2958936': 0.0,
 '2960384': 0.0,
 '2963372': 0.018660903361091757,
 '2963899': 0.0,
 '2965839': 0.0,
 '2967181': 0.0058540816387322812,
 '2983604': 0.0,
 '2985677': 0.0,
 '2989301': 0.0,
 '2989828': 0.0,
 '8066051': 0.0068541158019784186,
 'B00PWV9': 0.0,
 'B011WL6': 0.0,
 'B01R311': 0.0,
 'B067BM3': 0.0,
 'B08G9Z7': 0.0,
 'B0F0F09': 0.0032047995352079183,
 'B0J7D57': 0.0,
 'B0SRLF4': 0.0,
 'B0SRLH6': 0.0,
 'B12GHV2': 0.0,
 'B142VX8': 0.0,
 'B198380': 0.0,
 'B198391': 0.0,
 'B1F76F9': 0.0,
 'B1FLZ21': 0.0,
 'B1L60G9': 0.0,
 'B1Z77F6': 0.0064477249776569399,
 'B28V183': 0.0,
 'B3KFWW1': 0.0,
 'B3Q2FJ4': 0.0,
 'B40K911': -1.0842021724855044e-19,
 'B4NH4W5': 0.0,
 'B5BKPQ4': -4.3368086899420177e-19,
 'B5BT0K0': -1.0842021724855044e-19,
 'B633030': 0.0,
 'B63QTN2': 0.024764184316630678,
 'B6TFK30': 0.0,
 'B6WVMH3': 0.0,
 'B76FBQ0': 0.0,
 'B7QQYV9': -1.0842021724855044e-19,
 'B7VD3F2': 0.0,
 'B840261': 0.0,
 'B84LSN3': 0.0,
 'B8DMK08': 0.0,
 'B8KQN82': 0.002840515631730029,
 'B92RRW2': 0.0027665099287463722,
 'B9M2WX3': 0.0,
 'BBM4S68': 0.0,
 'BD5JS97': 0.0,
 'BD95VZ8': 0.0,
 'BDHLTQ5': 0.0058602583992211407,
 'BF0FM92': 0.0,
 'BF0HZC2': 0.0,
 'BF6V3T1': 0.0,
 'BFG3KF2': -1.0842021724855044e-19,
 'BJWG213': 0.0,
 'BLNN369': 0.0,
 'BMH4NJ8': 0.0027353827376411727,
 'BSPHGL4': 0.0,
 'BTN1Y11': 0.0,
 'BTN1Y44': -4.3368086899420177e-19,
 'BVFMFG8': 0.0022949955571392455,
 'BWSW5J5': 0.0,
 'BY7QL61': 0.0,
 'BY9D546': 0.0,
 'BYND5N1': 0.0,
 'BYND5W0': 0.0,
 'BYNF418': 0.0,
 'BYP66M4': 0.0,
 'BYQQ3P5': 0.0,
 'BYV2325': 0.0,
 'BYVY8G0': 0.0,
 'BYWTW73': 0.0,
 'BYX4D52': 0.0,
 'BYXB201': 0.0,
 'BYXD7B3': 0.0026008010829026286,
 'BYZGYC0': 0.0,
 'BZ0P3Z5': 0.0,
 'BZ2Y3W7': 0.0,
 'BZBY209': 0.0}

In [16]:
dfd

{'2000019': 0.0,
 '2002305': 0.0,
 '2002479': 0.0,
 '2005973': 0.0,
 '2007849': 0.00398966920824236,
 '2008154': 0.0,
 '2011602': 0.0,
 '2017213': 0.0,
 '2018175': 0.0,
 '2019479': 0.0,
 '2019952': 0.0,
 '2020459': 0.0,
 '2021690': 0.0,
 '2023607': 0.0,
 '2023737': 0.010691944567940037,
 '2023748': 0.0,
 '2024020': 0.0,
 '2026082': 0.0,
 '2026242': 0.0,
 '2026361': 0.0,
 '2027104': 0.0,
 '2027342': 0.0,
 '2028572': 0.0,
 '2029809': 0.0,
 '2032067': 0.0,
 '2032380': 0.0,
 '2033004': 0.0,
 '2034494': 0.0,
 '2034524': 0.0,
 '2038430': 0.0,
 '2039831': 0.0,
 '2041364': 0.0,
 '2043694': 0.0,
 '2043962': 0.0,
 '2044545': 0.0,
 '2046251': 0.018417262520140518,
 '2046552': 0.0,
 '2047317': 0.0,
 '2048804': 0.024950686485023615,
 '2050832': 0.0,
 '2057059': 0.0,
 '2065159': 0.0038942869231824717,
 '2065308': 0.0,
 '2065955': 0.0,
 '2066408': 0.0,
 '2066721': 0.0033493087018528125,
 '2068921': 0.0,
 '2069065': 0.0,
 '2073022': 0.0,
 '2073408': 0.0,
 '2077905': 0.0,
 '2081779': 0.0,
 '2084637': 0

In [15]:
sol_list = []
for k in risk_sedol :
    sol_list.append(sol[k])

con = select_candi(sol_list)
def set_con(cons):
    conlist = cons
    cplexs.set_con(sols=conlist)
set_con(cons=con)

NameError: name 'cplexs' is not defined

In [24]:
f0 = open('pic/dic_sector.txt', 'rb')
f1 = open('pic/dic_bench.txt', 'rb')
f2 = open('pic/risk_sedol.txt', 'rb')
f3 = open('pic/dic_MCAP.txt', 'rb')
f4 = open('pic/dic_beta.txt', 'rb')
f5 = open('pic/alpha.txt', 'rb')
f6 = open('pic/qmat.txt', 'rb')
f7 = open('pic/Q_con.txt', 'rb')
f8 = open('pic/risk_mat.txt', 'rb')

dic_sector = pickle.load(f0)
dic_bench = pickle.load(f1)
risk_sedol = pickle.load(f2)
dic_MCAP = pickle.load(f3)
dic_beta = pickle.load(f4)
alpha = pickle.load(f5)
qmat = pickle.load(f6)
risk_mat = pickle.load(f8)
omegamulti = 10000
conlist = [0,1,2,3,4,5]
q_con1 = []
q_con2 = []
q_val = []

for i in range(len((risk_mat[0]))):
    for j in range(len(risk_mat[0])):
        if j >= i:
            q_con1.append(i)
            q_con2.append(j)
            if i == j:
                ex_list = list(risk_mat[i])
                q_val.append(10000 * ex_list[j])
            else:
                ex_list = list(risk_mat[i])
                q_val.append(2*10000 * ex_list[j])

Q_cons = []
Q_cons.append(q_con1)
Q_cons.append(q_con2)
Q_cons.append(q_val)
Q_con = Q_cons
cn.portfolio(sector=dic_sector, bench=dic_bench, asset=risk_sedol, MCAPQ=dic_MCAP, beta=dic_beta,alpha=alpha)


In [17]:
import pickle

In [23]:
cplexs